# RBM training

In [ ]:
import numpy as np

from rbm import RBM

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

import torch
import torch.nn as nn
torch.manual_seed(12)

## Dataset with $ T = 1, 1.1, ..., 3.5$

Here we go.

## Dataset with $ T = 0$ and $ T = \infty$

In order to investigate the RBM flow in an extreme situation, we train the machine in an dataset composed only by frozen and random states. This still satisfies the condition shown in the work [Scale-invariant Feature Extraction of Neural Network and Renormalization Group Flow](https://arxiv.org/abs/1801.07172): the RBM flow would go towards the critical value if the data training set has states with temperature values less and greater than the critical one.

It is crucial to note that, since the visible layer is always fed with an unidimensional vector (an array is transformed in a vector to be fed in the visible layer), the machine does not have any geometric information about the lattice in this case.

### Case $T=0$ "UP/DOWN" and $ T = \infty$

#### Creating the dataset

Frozen configurations:

In [ ]:
n_states = 1000

training_lowT_up = nn.init.constant_(torch.empty(int(n_states/2), L*L),
                                     val= 1.0)

training_lowT_down = nn.init.constant_(torch.empty(int(n_states/2), L*L),
                                     val= 0.0)

training_set_ = torch.cat((training_lowT_down, training_lowT_up), 0)

Random configurations:

In [ ]:
training_highT = torch.empty(n_states, L*L).bernoulli_(p= 0.5)

Constructing and shuffling the training set:

In [ ]:
training_set = torch.cat((training_set_, training_highT), 0)

training_set = training_set[torch.randperm(training_set.size()[0])]

#### Training the model

For simplification, the units have no bias.

In [ ]:
Nv = training_set.shape[1]
Nh = training_set.shape[1]

lr = 0.001
k_learning = 1
batch_size = 100
nb_epoch = 448

rbm = RBM(num_visible= training_set.shape[1], 
          num_hidden= Nh, 
          bias= False,
          T= 1.0,
          use_cuda= True)
    
rbm.learn(training_set= training_set,  
          lr= lr, 
          nb_epoch= nb_epoch, 
          batch_size= batch_size,
          k_learning= k_learning)

#### Saving the trained model

In [ ]:
name = 'RBM_model_zero_inf_T__UP_DOWN_nv%d_nh%d_lr%.1E_k%d_bsize%d_nepochs%d' % (Nv,
                                                                                 Nh,
                                                                                 lr,
                                                                                 k_learning,
                                                                                 batch_size,
                                                                                 nb_epoch)

PATH = 'RBM_trained_models/'+ name + '.pt'

torch.save(rbm, PATH)

#### Weights distribution

In [ ]:
W, v, h = rbm.parameters()

In [ ]:
W_ = W.cpu().numpy().reshape((W.shape[0]*W.shape[1]))

# Plot normalized histogram 
plt.hist(W_, bins= 100, density= True)

# Maximum and minimum of xticks to compute the theoretical distribution 
x_min, x_max = min(plt.xticks()[0]), max(plt.xticks()[0])  
domain = np.linspace(x_min, x_max, len(W_))

# Fitting a normal distribution
muW_, sigmaW_ = stats.norm.fit(W_) 

plot_pdf = stats.norm.pdf(domain, muW_, sigmaW_) # Fitting the PDF in the interval


plt.plot(domain, plot_pdf, linewidth= 2.5,
         label= '$\mu= %f$ \n$\sigma^2$ = %f' % (muW_, sigmaW_**2 ))

plt.title('Fitting a Normal Distribution for the weights ${\cal W}$')
plt.legend()
plt.show()

### Case $T=0$ "UP" and $ T = \infty$

#### Creating the dataset

Frozen configurations:

In [ ]:
n_states = 1000

training_lowT_up = nn.init.constant_(torch.empty(n_states, L*L),
                                     val= 1.0)

Random configurations:

In [ ]:
training_highT = torch.empty(n_states, L*L).bernoulli_(p= 0.5)

Constructing and shuffling the training set:

In [ ]:
training_set = torch.cat((training_lowT_up, training_highT), 0)

training_set = training_set[torch.randperm(training_set.size()[0])]

#### Training the model

For simplification, the units have no bias.

In [ ]:
Nv = training_set.shape[1]
Nh = training_set.shape[1]

lr = 0.001
k_learning = 1
batch_size = 100
nb_epoch = 305

rbm = RBM(num_visible= training_set.shape[1], 
          num_hidden= Nh, 
          bias= False,
          T= 1.0,
          use_cuda= True)
    
rbm.learn(training_set= training_set,  
          lr= lr, 
          nb_epoch= nb_epoch, 
          batch_size= batch_size,
          k_learning= k_learning)

In [ ]:
name = 'RBM_model_zero_inf_T__UP_nv%d_nh%d_lr%.1E_k%d_bsize%d_nepochs%d' % (Nv,
                                                                            Nh,
                                                                            lr,
                                                                            k_learning,
                                                                            batch_size,
                                                                            nb_epoch)

PATH = 'RBM_trained_models/'+ name + '.pt'

torch.save(rbm, PATH)

#### Saving the trained model

In [ ]:
W, v, h = rbm.parameters()

In [ ]:
W_ = W.cpu().numpy().reshape((W.shape[0]*W.shape[1]))

# Plot normalized histogram 
plt.hist(W_, bins= 100, density= True)

# Maximum and minimum of xticks to compute the theoretical distribution 
x_min, x_max = min(plt.xticks()[0]), max(plt.xticks()[0])  
domain = np.linspace(x_min, x_max, len(W_))

# Fitting a normal distribution
muW_, sigmaW_ = stats.norm.fit(W_) 

plot_pdf = stats.norm.pdf(domain, muW_, sigmaW_) # Fitting the PDF in the interval


plt.plot(domain, plot_pdf, linewidth= 2.5,
         label= '$\mu= %f$ \n$\sigma^2$ = %f' % (muW_, sigmaW_**2 ))

plt.title('Fitting a Normal Distribution for the weights ${\cal W}$')
plt.legend()
plt.show()